## Анализ временых меток

Разархивируем данные

In [20]:
import os
import zipfile

source = "Final_parts"
target = "data"
filename = "parts.json"

os.makedirs(target, exist_ok=True)

with zipfile.ZipFile(source + ".zip", 'r') as zip_ref:
    zip_ref.extractall(target)

Преобразуем в архив с данными в список датафремов

In [21]:
import pandas as pd

games = []
root = os.path.join(target, source)

for dir in os.listdir(root):
    if dir == ".DS_Store": continue
    games.append(pd.read_json(os.path.join(root, dir, filename)))

print(len(games))

idx = 0
games[idx]

30


,move,start_ts,end_ts,end_of_move_ts,fragment_before_ts,fragment_after_ts,video_duration
0,e4,1744996940091,1744996942896,1744996942091,2000,805,2805
1,c6,1744996942896,1744996944443,1744996943701,805,742,1547
2,d4,1744996944443,1744996945659,1744996945185,742,474,1216
3,d5,1744996945659,1744996946805,1744996946133,474,672,1146
4,exd5,1744996946805,1744996947878,1744996947477,672,401,1073
...,...,...,...,...,...,...,...
69,Qe8,1744997356763,1744997358793,1744997358011,1248,782,2030
70,Rh3,1744997358794,1744997361576,1744997359576,782,2000,2782
71,Nh7,1744997362309,1744997365080,1744997364309,2000,771,2771
72,Qxh7+,1744997365081,1744997366415,1744997365852,771,563,1334


Приведем время к таймингам в самом видео, вычтя из каждого таймстемпа таймстемп начала партии

In [22]:
for game in games:
    game[["start_ts", "end_ts", "end_of_move_ts"]] -= game["start_ts"].iloc[0]

game = games[idx]
game

,move,start_ts,end_ts,end_of_move_ts,fragment_before_ts,fragment_after_ts,video_duration
0,e4,0,2805,2000,2000,805,2805
1,c6,2805,4352,3610,805,742,1547
2,d4,4352,5568,5094,742,474,1216
3,d5,5568,6714,6042,474,672,1146
4,exd5,6714,7787,7386,672,401,1073
...,...,...,...,...,...,...,...
69,Qe8,416672,418702,417920,1248,782,2030
70,Rh3,418703,421485,419485,782,2000,2782
71,Nh7,422218,424989,424218,2000,771,2771
72,Qxh7+,424990,426324,425761,771,563,1334


Проверим были ли вырезаны какие-то моменты из первого видео

In [23]:
(game["start_ts"].shift(-1).dropna() - game["end_ts"]).dropna().sort_values(ascending=False).head(30)

44    19832.0
23    19305.0
47    17507.0
42    16373.0
48    15579.0
34    12767.0
38    12108.0
18    11602.0
32    11203.0
36    10767.0
50     9236.0
51     7535.0
24     6463.0
49     6384.0
13     5855.0
37     5333.0
43     5284.0
67     4398.0
52     4147.0
20     3797.0
31     3470.0
40     2541.0
26     2443.0
58     2305.0
46     1800.0
22     1531.0
28     1043.0
70      733.0
19      318.0
60      228.0
dtype: float64

В видео было вырезано приличное количество моментов, поэтому будем ориентироваться на колонки ```start_ts``` и ```fragment_before_ts```

In [ ]:
source = "final_videos_prod_25_04_25"
target = "data"

In [34]:
os.makedirs(target, exist_ok=True)

with zipfile.ZipFile(source + ".zip", 'r') as zip_ref:
    zip_ref.extractall(target)

Нарезка видео по номеру хода

In [2]:
from slice_by_moves import extract_segments_by_move

out_dir = "out_data"
os.makedirs(out_dir, exist_ok=True)

extract_segments_by_move(
    "data/Final_parts/3067/parts.json",
    "data/final_videos_prod_25_04_25/3067/output.mp4",
    "out_data/video.mp4",
    [[0, 3], [4, 5], [10, 13]]
)

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf59.27.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [720, 1200], 'bitrate': 1069, 'fps': 30.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 48000, 'bitrate': 131, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 241.05, 'bitrate': 1208, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(High)', 'video_size': [720, 1200], 'video_bitrate': 1069, 'video_fps': 30.0, 'default_audio_input_number': 0, 

frame_index:   1%|          | 4/606 [00:07<20:52,  2.08s/it, now=None]

Proc not detected
{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf59.27.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [720, 1200], 'bitrate': 1069, 'fps': 30.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 48000, 'bitrate': 131, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 241.05, 'bitrate': 1208, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(High)', 'video_size': [720, 1200], 'video_bitrate': 1069, 'video_fps': 30.0, 'default_audio_

frame_index:   6%|▌         | 34/606 [01:20<24:47,  2.60s/it, now=None]

KeyboardInterrupt: 